# 1- Problem Description

Almadinah city in Saudi Arabia has the second holey mosque for Muslims around the world. Because of that it is visited during a whole year.  Therefore, it is good for visitors to know the neighborhoods of the city and its characteristics. This will lead them to book their hotels and stay during their visits in their favorite neighborhoods.  As a result, in this project I am aiming to cluster the neighborhoods in the city of Almadinah  using  k-means clustering technique. 

# 2- Objective

The main goal of this project is to cluster the neighborhoods in the city of Almadinah in Saudi Arabia. For achieving that the neighborhoods of Almadinah city will be studied and analyzed first. Then, K-means is implemented. This will assist visitors of the city to know where to stay in the city during their visit. 

# 3- Target Audience

The results of this project would be helpful for people who are interested in visiting Almadinah city and knowing its neighborhoods. 

# 4- Data Description

### 4.1- Location Data of Saudi Arabia

This project on github (https://github.com/homaily/Saudi-Arabia-Regions-Cities-and-Districts/tree/master/json) has information about Regions, Cities and Districts in Saudi Arabia. The data is public and collected from https://maps.address.gov.sa/ as ,all coordinates in (Lat, Lon) aka (Y, X) format and 8 decimal points. Data points include: Regions, Cities and Districts. All names are written in tow languages: Arabic and English. This data is divided into three parts:
 - regions.json
 - cities.json
 - districts.json
 
However, the first step is to covert  this data into a pandas data frame for further analysis.


### 4.2- Coordinate data for Saudi Cities

All the venues in each neighborhood in the KSA cities will be collected from Fousquare using Fousquare API.  Because we are interested in neighborhoods of Almadinah city, therefore, its  venues will be considered. 

### 4.3- Sample from data and features

In [8]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np # library to handle data in a vectorized manner


import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

import json # library to handle JSON filesasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasasas

Libraries imported.


#### Loading regions.json for extract KSA regions and explore it

In [ ]:
with open('regions.json', encoding="utf-8") as json_data:    
    country_data = json.load(json_data)
print('No of regon:',len(country_data) )
country_data

#### Then convert json object of KSA regions to Pandas dataframe

In [12]:
country_regions= pd.DataFrame(columns= [ 'region_id', 'name_ar', 'name_en','population' , 'capital_city_id','Latitude', 'Longitude'] )

for i,data in enumerate (country_data):
    #print(i, end=' ')
    row=[]
    for c in country_regions.columns:
        if c == 'Latitude' :
            row.append(data['center'][0])    
        elif c== 'Longitude':  
            row.append(data['center'][1]) 
        else:
            row.append(data[c])    
    #print(row)
    country_regions.loc[len(country_regions)] = row
    
country_regions.head()

,region_id,name_ar,name_en,population,capital_city_id,Latitude,Longitude
0,1,منطقة الرياض,Riyadh,6777146,3,24.70000,46.73333
1,2,منطقة مكة المكرمة,Makkah,6915006,6,21.42718,39.84349
2,3,منطقة المدينة المنورة,Madinah,1777933,14,24.47058,39.60781
3,4,منطقة القصيم,Qassim,1215858,11,26.33034,43.97436
4,5,المنطقة الشرقية,Eastern Province,4105780,13,26.44199,50.10920


#### Loading cities.json for extract KSA cities and explore it

In [19]:
with open('cities.json', encoding="utf-8") as json_data:    
    cities_data = json.load(json_data)

#### Then convert json object of KSA cities to Pandas dataframe

In [21]:
country_cities= pd.DataFrame(columns= ['city_id', 'region_id', 'name_ar', 'name_en','Latitude', 'Longitude'] )


for data in cities_data:
    row=[]
    for c in country_cities.columns:
        if c == 'Latitude' :
            row.append(data['center'][0])    
  
        elif c== 'Longitude':
  
            row.append(data['center'][1]) 
        else:
            row.append(data[c])    
    #print(row)
    country_cities.loc[len(country_cities)] = row
    
country_cities.head()

,city_id,region_id,name_ar,name_en,Latitude,Longitude
0,1,7,تبوك,Tabuk,28.41464,36.53387
1,2,7,نعمي,Na'mi,28.30508,35.74931
2,3,1,الرياض,Riyadh,24.70000,46.73333
3,4,7,حميط,Humayt,28.65152,35.38013
4,5,2,الطائف,At Taif,21.26848,40.41667


#### Loading cities.json for extract KSA districts and explore it

In [24]:
#with open('newyork_data.json') as json_data:
with open('districts.json', encoding="utf-8") as json_data:    
    districts_data = json.load(json_data)

#### Then convert json object of KSA districts to Pandas dataframe

In [31]:
country_districts= pd.DataFrame(columns= ['district_id','city_id', 'region_id', 'name_ar', 'name_en','Latitude', 'Longitude'] )

for data in districts_data:
    row=[]
    for c in country_districts.columns:
        if c == 'Latitude' :
            b = np.array(data['boundaries'][0])
            b=b.mean(axis=0 )
            row.append(b[0])   
            row.append(b[1])    
        elif c == 'Longitude' :
            None
        else:
            row.append(data[c])    
        #print(row)
    
    country_districts.loc[len(country_districts)] = row
    
    
country_districts.head()

,district_id,city_id,region_id,name_ar,name_en,Latitude,Longitude
0,10100003001,3,1,حي العمل,Al Amal Dist.,24.644966,46.723598
1,10100003002,3,1,حي النموذجية,Al Namudhajiyah Dist.,24.655615,46.696523
2,10100003003,3,1,حي الجرادية,Al Jarradiyah Dist.,24.618815,46.696110
3,10100003004,3,1,حي الصناعية,Al Sinaiyah Dist.,24.646447,46.741675
4,10100003005,3,1,حي منفوحة الجديدة,Manfuha Al Jadidah Dist.,24.613354,46.716463
